In this tutorial we will see what it takes to load and render an .obj model using the pyGanadalf API.
-----------------------------------------------------------------------------------------------------

Before doing anything else we need to import the nessasary packages

In [1]:
from pyGandalf.core.application import Application
from pyGandalf.core.opengl_window import OpenGLWindow
from pyGandalf.systems.transform_system import TransformSystem
from pyGandalf.systems.camera_system import CameraSystem
from pyGandalf.systems.light_system import LightSystem
from pyGandalf.systems.opengl_rendering_system import OpenGLStaticMeshRenderingSystem

from pyGandalf.renderer.opengl_renderer import OpenGLRenderer

from pyGandalf.scene.scene import Scene
from pyGandalf.scene.components import *
from pyGandalf.scene.scene_manager import SceneManager

from pyGandalf.utilities.opengl_material_lib import OpenGLMaterialLib, MaterialData
from pyGandalf.utilities.opengl_texture_lib import OpenGLTextureLib, TextureData
from pyGandalf.utilities.opengl_shader_lib import OpenGLShaderLib
from pyGandalf.utilities.mesh_lib import MeshLib

from pyGandalf.utilities.definitions import SHADERS_PATH, MODELS_PATH

import numpy as np
import glm

First, we create a new application. To achieve this, we need to specify two main things:
- The window
  - We create an `OpenGLWindow` and we specify the name, width, height and vertical sync mode accordingly.
- The renderer
  - We specify that we will use the `OpenGLRenderer`.

In [2]:
Application().create(OpenGLWindow('Model Loading', 1280, 720, True), OpenGLRenderer)

Then, we build a texture, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLTextureLib` we call the `build` method to build our shader.
- 2. We give our texture a name, in this case we chose: "white_texture".
- 3. We provide the build function with the data and descriptor of the texture:
  - TextureData: Holds the data of the texture. You can either give a path (or list of paths if cubemap) or the byte data to use when creating the texture and the width and height.
  - TextureDescriptor: The description of the texture, which consists of various options and flags. Here, we dont set any options.
  
  In this example, we dont use an existing image as a texture, but we create one from the given bytes. More specifically, we create a 1 by 1 all white texture.

In [3]:
OpenGLTextureLib().build('white_texture', TextureData(image_bytes=0xffffffff.to_bytes(4, byteorder='big'), width=1, height=1))

0

Then, we build a shader, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLShaderLib` we call the `build` method to build our shader.
- 2. We give our shader a name, in this case we chose: "lit_blinn_phong".
- 3. We provide the build function with the path to the source code of the shader stages that we want our shader to utilize, in this case we use a vertext and fragment shader.
    
    In this example we create a simple shader that implements the Blinn-Phong shading method to shade the model that we will load.

In [4]:
OpenGLShaderLib().build('lit_blinn_phong', SHADERS_PATH / 'opengl' / 'lit_blinn_phong.vs', SHADERS_PATH / 'opengl' / 'lit_blinn_phong.fs')

3

Then, we build a material, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLMaterialLib` we call the `build` method to build our material.
- 2. We give the material a name, in this case we chose: "M_Bunny".
- 3. We give specify the desired data of the material through the `MaterialData` class.
  - As the first argument, we give them shader name that we want to use, we give the name of the shader we created above, meaning: "lit_blinn_phong".
  - The second argument is used to define which textures the material we use, in our case we use the 'white_texture' that we created before.

In [5]:
OpenGLMaterialLib().build('M_Bunny', MaterialData('lit_blinn_phong', ['white_texture']))

Now its time to load our model, lets take a closer look at how this can be achieved.
- 1. From the singleton class `MeshLib`, we call the `build` method to build our mesh.
- 2. We give the mesh a name, in this case we chose: "bunny_mesh".
- 3. Then, we specify the path of the mesh that we want to load.

In [6]:
MeshLib().build('bunny_mesh', MODELS_PATH / 'bunny.obj')

We create a new scene by instatiating an object of the class `Scene`, we give it the name "Bunny".

In [7]:
scene = Scene('Model Loading')

Then, we enroll a new entity to the scene by calling the `enroll_entity` method. This entity will represent our bunny.

In [8]:
bunny = scene.enroll_entity()

Now, its time to add components to our newly created entity, i.e.: `bunny`.

First, we using our `scene` and its `add_component` method, we add to the `triangle` entity, a `TransformComponent`.

We when are creating the `TransformComponent`, we specify the translation, rotation and scale vectors.

In [9]:
scene.add_component(bunny, TransformComponent(glm.vec3(0, 0, 0), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, we want to add a `StaticMeshComponent` component to the `bunny` entity.
- We add the `StaticMeshComponent` specifying the name of the mesh that we want to load, we choose the "bunny_mesh" that we loaded above.
- We dont need to specify any mesh data since they will all be retrieved from the mesh.

In [10]:
scene.add_component(bunny, StaticMeshComponent('bunny_mesh'))

Next, we have to add the `MaterialComponent`.

We just have to give the name of the material we have built before, i.e.: "M_Bunny".

In [11]:
scene.add_component(bunny, MaterialComponent('M_Bunny'))

Now, lets create a new entity in our scene, using the `enroll_entity` method, for the camera.

In [12]:
camera = scene.enroll_entity()

Then, using our `scene` and its `add_component` method, we add to the `camera` entity, a `TransformComponent`.

We position the camera 3 meters back and 1 meters up.

In [13]:
scene.add_component(camera, TransformComponent(glm.vec3(0, 1, 4), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, using our `scene` and its `add_component` method, we add to the `camera` entity, a `CameraComponent`.

We provide the `CameraComponent` with the required data: field of view, aspect ratio, near plane, far plane and the type `CameraComponent.Type.PERSPECTIVE` or `CameraComponent.Type.ORTHOGRAPHIC`

In [14]:
scene.add_component(camera, CameraComponent(45, 1.778, 0.1, 1000, 1.2, CameraComponent.Type.PERSPECTIVE))

Now, lets create a new entity in our scene, using the `enroll_entity` method, for the light.

In [15]:
light = scene.enroll_entity()

Using our `scene` and its `add_component` method, we add to the `light` entity, a `TransformComponent`.

In [16]:
scene.add_component(light, TransformComponent(glm.vec3(0, 4, 0), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, using our `scene` and its `add_component` method, we add to the `light` entity, a `LightComponent`.

We provide the `CameraComponent` with the required data: the color and the intensity.

In [17]:
scene.add_component(light, LightComponent(glm.vec3(1.0, 1.0, 1.0), 0.75))

Now that we have the components added to the entities, we have to register systems that we are gonna use to the scene.
In this example we are using two systems:
- `TransformSystem`
- `CameraSystem`
- `LightSystem`
- `OpenGLStaticMeshRenderingSystem`

When creating a system we have to provide to the constructor an array that holds the component(s) that the system operates on.

Then the system uses these component(s) to filter all the entities that have them.
- For example, the `TransformSystem` operates on all the entities that have a `TransformComponent`.
  - The `LightSystem` operates on all the entities that have a `LightComponent` and a `TransformComponent`.
  - Similarly the `OpenGLStaticMeshRenderingSystem` operates on all the entities that have a `StaticMeshComponent`, a `MaterialComponent` and a `TransformComponent`.
  - Finally, the `CameraSystem` operates on all the entities that have a `CameraComponent` and a `TransformComponent`.

We can register them to the scene by calling the `register_system` method.

In [18]:
scene.register_system(TransformSystem([TransformComponent]))
scene.register_system(CameraSystem([CameraComponent, TransformComponent]))
scene.register_system(LightSystem([LightComponent, TransformComponent]))
scene.register_system(OpenGLStaticMeshRenderingSystem([StaticMeshComponent, MaterialComponent, TransformComponent]))

Now that our scene has all the entites, components and systems added we can add it to the `SceneManager` by calling the `add_scene` method.

In [19]:
SceneManager().add_scene(scene)

Finally, we can start the `Application` by calling the `start` method.

In [20]:
Application().start()